In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from tqdm import tqdm
from torchvision import datasets,models,transforms
from torch.utils.data import DataLoader

In [2]:
device = torch.device("cuda")
device

device(type='cuda')

In [3]:
data_transforms = {
   "train":transforms.Compose([
       transforms.RandomResizedCrop(224),
       transforms.RandomHorizontalFlip(),
       transforms.ToTensor(),
       transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
   ]),
    "test":transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    
}

In [4]:
data_dir = "/home/vulcan/Documents/melanoma_cancer_dataset"

In [5]:
image_datasets = {x: datasets.ImageFolder(f"{data_dir}/{x}",data_transforms[x]) for x in ['train','test']}

In [6]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],batch_size=4,shuffle=True,num_workers=0) for x in ['train','test']}

In [7]:
model = models.resnet50(pretrained=True)
model = model.to(device)

/home/vulcan/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/vulcan/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:

num_fltrs = model.fc.in_features
models.fc =  nn.Linear(num_fltrs,28)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.001)



In [9]:
train_loss_list = []
train_acc_list = []
val_lost_list = []
val_acc_list = []

num_epochs = 25

for epoch in tqdm(range(num_epochs)):
    for phase in ["train","test"]:
        if phase =="train":
            model.train()
        else:
            model.eval()
        running_loss = 0.0
        corrects = 0

        for inputs,labels in dataloaders[phase]:
            optimizer.zero_grad()
            inputs,labels = inputs.to(device), labels.to(device)
            
            with torch.set_grad_enabled(phase=="train"):
                outputs = model(inputs)
                loss = criterion(outputs,labels)
                
                if phase == "train":
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _,preds = torch.max(outputs,1)
            corrects += torch.sum(preds==labels.data)

        epochs_loss = running_loss / len(image_datasets[phase])
        epoch_acc = corrects.double() / len(image_datasets[phase])

        print(f'{phase} Loss: {epochs_loss:.4f}, Acc:{epoch_acc:.4f}')
                

  0%|                                                    | 0/25 [00:00<?, ?it/s]/home/vulcan/.local/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


train Loss: 0.3259, Acc:0.8899


  4%|█▋                                       | 1/25 [02:56<1:10:29, 176.21s/it]

test Loss: 0.0260, Acc:0.9909
train Loss: 0.1916, Acc:0.9320


  8%|███▎                                     | 2/25 [05:51<1:07:15, 175.45s/it]

...

test Loss: 0.0208, Acc:0.9919
train Loss: 0.0816, Acc:0.9680


 96%|██████████████████████████████████████▍ | 24/25 [1:11:32<02:59, 179.05s/it]

test Loss: 0.0213, Acc:0.9920
train Loss: 0.0792, Acc:0.9671


100%|████████████████████████████████████████| 25/25 [1:14:29<00:00, 178.77s/it]

test Loss: 0.0189, Acc:0.9925


In [10]:
torch.save(model, 'cancer.pt')